# 1)

In [49]:
import numpy as np
import IPython
from scipy.io import wavfile

# Tone genrating function
def ToneSin(o=[]):
    f=8000
    t=0.95
    points=int(t*f)                # Code Relies on number of points which are calculated using the frequency and time 
    #print(points)
    x=[0]*points
    y=[0]*points
    z=[0]*points
    
    # Checking for which number tone to generate which assigns the two frequencies associated with the number 
    if o=="1":
        m=1209
        p=697
    elif o=="2":
        m=1336
        p=697
    elif o=="3":
        m=1477
        p=697
    elif o=="4":
        m=1209
        p=770
    elif o=="5":
        m=1336
        p=770
    elif o=="6":
        m=1477
        p=770
    elif o=="7":
        m=1209
        p=852
    elif o=="8":
        m=1336
        p=852
    elif o=="9":
        m=1477
        p=852
    elif o=="*":
        m=1209
        p=941
    elif o=="0":
        m=1336
        p=941
    elif o=="#":
        m=1477
        p=941
    #print(m,p)
    
    # Assigned frequencies caclculate the "omega not" values
    w=2*np.pi*m/f
    g=2*np.pi*p/f
    
    # A is used to increase the intensity of the tone to be able to hear it
    A=10000
    
    # x and y are both output arrays of the difference equation for the two difference frequencies in the dual tone
    
    y[0]=0
    y[1]=np.sin(w)
    x[0]=0
    x[1]=np.sin(g)
    
    for i in range(2,points):
        y[i]=2*np.cos(w)*y[i-1]-y[i-2]
        x[i]=2*np.cos(g)*x[i-1]-x[i-2]
        
        # z is the final array with the dual tone signal 
        z[i]=A*(y[i]+x[i])
        #print(y[i],x[i],z[i]/1000)
    
    return z


# Testing the function and generating KPU's phone number
fs=8000
t1=0.95
points1=int(t1*fs)

f123="1"
u=ToneSin(f123)
#print(len(u))


k="6045992000"
kpuNumber=[0]*(points1*10)
g=[0]*(points1*10)

i=0
s=-points1
for i in range(0,10):
    j=0
    g=ToneSin(k[i])
    s+=points1
    #print(k[i])
    for j in range(0,points1):
        #print(s+j)
        kpuNumber[s+j]=g[j]


Tone1 = np.asarray(kpuNumber, dtype=np.int16)
wavfile.write('Tone1.wav',fs,Tone1)

IPython.display.Audio("Tone1.wav") 


# Tried this on KPU line and it was used to navigate through.





# 2)

In [50]:
import matplotlib.pyplot as plt
from scipy import signal

# The function has been adjusted to operate with a tone length t>=0.05 seconds. When "no signal" or too short of a signal
# is sent, the function returns "No Tone"


def BandPass(x,hi_cutoff, low_cutoff):
    fs=8000
    nyq = 0.5 * fs
    normal_cutoff = hi_cutoff / nyq
    b, a = signal.butter(8, normal_cutoff, btype='high', analog=False)
    output = signal.filtfilt(b, a, x)
    
    nyq = 0.5 * fs
    normal_cutoff = low_cutoff / nyq
    b, a = signal.butter(8, normal_cutoff, btype='low', analog=False)
    output = signal.filtfilt(b, a, output)
    
    return output


def Recons(list1):
    length=len(list1)
    sum1=[0]*4           # list to store sum of filtered list
    sum2=[0]*4
    
    # Checking for Column frequencies
    # Adding the absolute value of the filtered signal provides ensures that the magnitude is what we are expecting
    # for the unfiltered frequency dual tone to be largest
    filt1=BandPass(list1,1190,1225)
    sum1[0]=sum(np.abs(filt1[0:length]))/10000

    filt2=BandPass(list1,1300,1350)
    sum1[1]=sum(np.abs(filt2[0:length]))/10000
    
    filt3=BandPass(list1,1450,1500)
    sum1[2]=sum(np.abs(filt3[0:length]))/10000
    
    # argmax is a numpy function I found online which tells which index has the largest value 
    
    h=np.argmax(sum1)
    #print(sum1[h])
    
    #Threshold value of the column frequencies was much different from the row so below threshold sets a variable value 
    # to 3 which is shown to relate to the output number array
    if sum1[h]< 79:
        h=3
    
    # Checking for Row frequencies
    filt4=BandPass(list1,650,710)
    sum2[0]=sum(np.abs(filt4[0:length]))/10000
        
    filt5=BandPass(list1,750,800)
    sum2[1]=sum(np.abs(filt5[0:length]))/10000
    
    filt6=BandPass(list1,820,870)
    sum2[2]=sum(np.abs(filt6[0:length]))/10000
    
    filt7=BandPass(list1,920,960)
    sum2[3]=sum(np.abs(filt7[0:length]))/10000
    
    
    r=np.argmax(sum2)
    #print(sum2[r])
    
    # Threshold for the row frequencies
    if sum2[r]< 86.5:
        r=0
    
    # The array of keypad outputs of this function
    Tone=[[1,4,7,'*'],[2,5,8,0],[3,6,9,'#'],['No Tone']] 
    
    return Tone[h][r] 

# Testing the function

g123="5"
u12=ToneSin(g123)
toneee=Recons(u12)  
print(toneee)




5
